In [11]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
import pickle
import plotly.express as px

In [5]:
train_data = pd.read_excel('Flight Dataset/Data_Train.xlsx')
train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


# Destination Analysis

In [6]:
train_data['Destination'].value_counts()

Cochin       4537
Banglore     2871
Delhi        1265
New Delhi     932
Hyderabad     697
Kolkata       381
Name: Destination, dtype: int64

**Delhi** and **New Delhi** are basically the same place. Let's join them together first.

In [7]:
def newd(x):
    if x=='New Delhi':
        return 'Delhi'
    else:
        return x

train_data['Destination'] = train_data['Destination'].apply(newd)

In [9]:
destination_counts = train_data['Destination'].value_counts(); destination_counts

Cochin       4537
Banglore     2871
Delhi        2197
Hyderabad     697
Kolkata       381
Name: Destination, dtype: int64

In [31]:
#converting destination_counts to pandas dataframe to use for plotting
destination_counts_df = pd.DataFrame(destination_counts,)
destination_counts_df['Destination_pct'] =  round((destination_counts_df['Destination'] / destination_counts_df['Destination'].sum()) * 100, 2)


In [32]:
destination_counts_df

,Destination,Destination_pct
Cochin,4537,42.47
Banglore,2871,26.87
Delhi,2197,20.57
Hyderabad,697,6.52
Kolkata,381,3.57


In [33]:
fig = px.bar(destination_counts_df, y='Destination_pct', labels={
                     "index": "Destination",
                     "Destination_pct": "Count(%)", 
                 })
fig.show()

Maximum people are going to Cochin followed by Bangalore and then Delhi in our dataset. So, the top 3 destinations are:
- Cochin(42%)
- Bangalore(27%)
- Delhi(20%)

Kolkata receives the least traffic.